# Using TensorBoard in Red Hat OpenShift AI Workbenches

Run and open TensorBoard to monitor the training process.

### 1. Import the required libraries and packages.

You can safely ignore the TensorFlow import warnings.
TensorFlow typically produces these warnings on CPU-only environments where the use of accelerators, such as GPUs or certain CPU instruction sets, is limited or not available.

In [7]:
import os
import datetime
from typing import List, Dict

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

### 2. Load and Preprocess the Data

Split the data into train, test, and validation subsets.

In [9]:
data = pd.read_csv('./data/diabetes.csv')

X = data.drop('Outcome', axis=1)
y = data['Outcome']

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=0
)

X_test, X_validation, y_test, y_validation = train_test_split(
    X_test,
    y_test,
    test_size=0.3,
    random_state=0
)

print(f"Number of samples in training set: {X_train.shape[0]}")
print(f"Number of samples in test set: {X_test.shape[0]}")

Number of samples in training set: 537
Number of samples in test set: 161


### 4. Create and train the model.

Define the model.
Use `accuracy` as the metric to evaluate the model.

In [12]:
# Seed for reproducible results
tf.random.set_seed(10)
tf.keras.utils.set_random_seed(10)

model = tf.keras.Sequential([
    tf.keras.layers.Input((8,)),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

Define the logging callback.

In [18]:
log_dir = "logs/training/" + datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
logging_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

Train the model.
Use `validation_data` to validate the model after each epoch.

TensorFlow reports the value of the loss function and the `accuracy` metric for both the training and the validation subsets.

In [19]:
model.fit(
    X_train,
    y_train,
    epochs=50,
    validation_data=(X_validation, y_validation),
    callbacks=[logging_callback]
)

Epoch 1/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7831 - loss: 0.4637 - val_accuracy: 0.8714 - val_loss: 0.4195
Epoch 2/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7723 - loss: 0.4614 - val_accuracy: 0.8714 - val_loss: 0.4237
Epoch 3/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7830 - loss: 0.4639 - val_accuracy: 0.8286 - val_loss: 0.4283
Epoch 4/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7831 - loss: 0.4589 - val_accuracy: 0.8714 - val_loss: 0.4176
Epoch 5/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7705 - loss: 0.4670 - val_accuracy: 0.8429 - val_loss: 0.4426
Epoch 6/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7795 - loss: 0.4597 - val_accuracy: 0.8286 - val_loss: 0.4387
Epoch 7/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7828 - loss: 0.4615 - val_accuracy: 0.8571 - val_loss: 0.4336
Epoch 8/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7685 - loss: 0.4644 - val_accuracy: 0.8286 - val_loss:

### 5. Run TensorBoard

Load the `tensorboard` notebook extension.

In [16]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Set the `TENSORBOARD_PROXY_URL` environment variable to access TensorBoard from this notebook.

In [ ]:
os.environ["TENSORBOARD_PROXY_URL"] = os.environ["NB_PREFIX"]+"/proxy/6006/"

Run the TensorBoard server.

In [17]:
%tensorboard --logdir logs/training